In [ ]:
from google.colab import files
files.upload()

Saving archive.zip to archive.zip


In [ ]:
import zipfile
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [6]:
import os
print(os.listdir('.'))

['.config', 'archive.zip', 'test', 'train', 'sample_data']


## **Dataset information**
The dataset contain 35,685 examples of 48x48 pixel gray scale images of faces divided into train and test dataset. 
Images are categorized based on the emotion shown in the facial expressions:

*   hapiness
*   neutral
*   sadness
*   anger
*   surprise
*   disgust
*   fear

In [8]:
from __future__ import print_function
import keras
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Add
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np


batch_size = 256
num_classes = 7
epochs = 50

train_dir = 'train'
test_dir = 'test'


class CustomGenerator():
    def __init__(self, generator, directory, batch_size, alpha=0.3):
        self.batch_index = 0
        self.batch_size = batch_size

        self.generator_X = generator.flow_from_directory(directory,
                                                        target_size=(48, 48),
                                                        class_mode="categorical",
                                                        color_mode='grayscale',
                                                        batch_size=batch_size,
                                                        shuffle=True)
        
        self.generator_Y = generator.flow_from_directory(directory,
                                                        target_size=(48, 48),
                                                        class_mode="categorical",
                                                        color_mode='grayscale',
                                                        batch_size=batch_size,
                                                        shuffle=True)
        self.n = self.generator_X.samples
        self.alpha = alpha

    def reset_index(self):
        self.generator_X._set_index_array()
        self.generator_Y._set_index_array()

    def on_epoch_end(self):
        self.reset_index()

    def reset(self):
        self.batch_index = 0

    def __len__(self):
        return (self.n + self.batch_size - 1) // self.batch_size

    def get_steps_per_epoch(self):
        return self.n // self.batch_size

    def __next__(self):
        if self.batch_index == 0:
            self.reset_index()

        current_index = (self.batch_index * self.batch_size) % self.n
        if self.n > current_index + self.batch_size:
            self.batch_index += 1
        else:
            self.batch_index = 0
        X1, y1 = self.generator_X.next()
        X2, y2 = self.generator_Y.next()

        l = np.random.beta(self.alpha, self.alpha, X1.shape[0])
        X_l = l.reshape(X1.shape[0], 1, 1, 1)
        y_l = l.reshape(X1.shape[0], 1)

        X = X1 * X_l + X2 * (1 - X_l)
        y = y1 * y_l + y2 * (1 - y_l)
        return X, y

    def __iter__(self):
        while True:
            yield next(self)


datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

train_dataset = CustomGenerator(datagen, train_dir, batch_size)

test_dataset = datagen.flow_from_directory(test_dir,
                                           target_size=(48, 48),
                                           class_mode="categorical",
                                           color_mode='grayscale',
                                           batch_size=batch_size,
                                           shuffle=True)

Found 28709 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [9]:
# Definición de la red
def resnet_layer(inputs, filters, apply_stride=False):
    x = inputs

    if apply_stride:
      x = Conv2D(filters, (3, 3), strides=2, padding='same')(x)
    else:
      x = Conv2D(filters, (3, 3), padding='same')(x)

    x = BN()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BN()(x)

    if apply_stride:
      previous = Conv2D(filters, (1,1), strides=2, padding='same')(inputs)
    else:
      previous = Conv2D(filters, (1,1), padding='same')(inputs)

    x = Add()([previous,x])
    x = Activation('relu')(x)

    return x

inputs = Input(shape=(48, 48, 1))

x = resnet_layer(inputs, 128)
x = resnet_layer(x, 128)

x = resnet_layer(x, 256, True)
x = resnet_layer(x, 256)


x = resnet_layer(x, 512, True)
x = resnet_layer(x, 512)


x = GlobalAveragePooling2D()(x)

x = Dense(num_classes)(x)
x = Activation('softmax')(x)

model = Model(inputs=inputs, outputs=x)
model.summary()


## OPTIM AND COMPILE
opt = Adam(learning_rate=0.001)


model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.000001)

## TRAINING with DA and LRA
history = model.fit(train_dataset,
                    epochs = epochs,
                    steps_per_epoch= 28709/ batch_size,
                    callbacks=[reduce_lr],
                    validation_data=test_dataset,
                    verbose=1)

## TEST
scores = model.evaluate(test_dataset, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 conv2d_18 (Conv2D)             (None, 48, 48, 128)  1280        ['input_2[0][0]']                
                                                                                                  
 batch_normalization_12 (BatchN  (None, 48, 48, 128)  512        ['conv2d_18[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_13 (Activation)     (None, 48, 48, 128)  0           ['batch_normalization_12[0]